In [9]:
import torch
import torch.nn as nn
import numpy as np

In [10]:
def print_model_performance(outputs: torch.tensor):
    outputs = outputs.detach().numpy().flatten()
    print(f'Outputs: {outputs}')
    output_sum = np.sum(outputs)
    print(f'Output sum: {output_sum}')
    delta_output = outputs[0] + outputs[2] - outputs[1] - outputs[3]
    print(f'Delta output: {delta_output}')

In [11]:
class SimpleNet(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleNet, self).__init__()
        h_size = 32
        self.fc1 = nn.Linear(input_size, h_size)
        # self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(h_size, h_size)
        self.fc4 = nn.Linear(h_size, h_size)
        self.fc5 = nn.Linear(h_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        # x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [12]:
model_path = 'models/TA_optimization_NN.pth'
model = SimpleNet(6, 4)
model_dict = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(model_dict)
model.eval()

SimpleNet(
  (fc1): Linear(in_features=6, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=4, bias=True)
  (relu): ReLU()
)

In [13]:
layer_names = [name for name in model_dict.keys()]
weights = [model_dict[name].detach().cpu().numpy() for name in layer_names if 'weight' in name]
biases = [model_dict[name].detach().cpu().numpy() for name in layer_names if 'bias' in name]

In [14]:
input_values = torch.tensor([500, -200, 100, 100, 200, 200], dtype=torch.float32)
output_values = model(input_values)
print_model_performance(output_values)

Outputs: [ 24.905003 100.47629   75.65042  200.22124 ]
Output sum: 401.2529296875
Delta output: -200.14210510253906


In [15]:
# Perform the same calculation manually
input_values_numpy = input_values.detach().numpy()
h1 = weights[0] @ input_values_numpy + biases[0]
h1 = np.maximum(h1, 0)
h2 = weights[1] @ h1 + biases[1]
h2 = np.maximum(h2, 0)
h3 = weights[2] @ h2 + biases[2]
h3 = np.maximum(h3, 0)
output = weights[3] @ h3 + biases[3]
print_model_performance(torch.tensor(output))

Outputs: [ 24.905003 100.4763    75.650406 200.22124 ]
Output sum: 401.2529296875
Delta output: -200.1421356201172


In [16]:
# Save the weights and biases to a file
directory = 'models/manual_weights/'
for i, (weight, bias) in enumerate(zip(weights, biases)):
    np.savetxt(directory + f'weight_{i}.csv', weight, delimiter='\t')
    np.savetxt(directory + f'bias_{i}.csv', bias, delimiter='\t')    